## Import your stuff

In [ ]:
import numpy as np,numpy.linalg
import emcee
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import matplotlib as mpl
import simulator as faas
import pydelfi.ndes as ndes
import pydelfi.delfi as delfi
import pydelfi.score as score
import pydelfi.priors as priors
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline
seed = 1

## Set up the prior

In [ ]:
upper = np.array([12,  12,
  12,  12, -2, -2, -2,
 -2])
lower = np.array([6,  6,
  6,  6, -8, -8, -8,
 -8])

prior_mean = np.array([1.1e-03, -3.9e-01])
prior_cov = np.diag(np.array([(0.0011*0.2)**2, (0.39*0.2)**2]))

prior = priors.Gaussian_Unif(prior_mean, prior_cov, lower, upper, 2)

epsilon_mean = np.array([0]*94)
epsilon_cov = np.diag(np.array([0.5] * 94))
epsilon_prior = priors.Gaussian(epsilon_mean, epsilon_cov)

#print(faasSimulator.exp(1))

## Set up the simulator
This must have the signature `simulator(parameters, seed, args, batch)` -> `np.array([batch, ndata])`

In [ ]:
faasSimulator = faas.faas_Model()

## repeating last fratio for short rows
def simulator(theta, seed, simulator_args, batch):
    
    eps_prior = simulator_args[0]
    eps = eps_prior.draw()
    #eps = [0] * 94
    
    
    return faasSimulator.simulation(np.concatenate([theta, eps]), seed)

simulator_args = [epsilon_prior]
theta0 = faasSimulator.theta0

theta_f = [8.886491e+00,  7.924279e+00,
   1.050515e+01,  7.397940e+00, -3.682371e+00, -4.509306e+00, -6.162727e+00,
  -6.585027e+00,  1.100000e-03, -3.900000e-01]

# th = [ 9.00645921e+00,  7.92686304e+00,  1.04450893e+01,  7.42039955e+00,
#  -3.63173724e+00, -4.50920711e+00, -6.22102446e+00, -6.60902990e+00,
#   1.09551239e-03, -3.87978817e-01]
# print(simulator(theta, 0, simulator_args, 1).shape)


In [ ]:
# theta_fiducial = np.array([8.886491e+00,  7.924279e+00,
#   1.050515e+01,  7.397940e+00, -3.682371e+00, -4.509306e+00, -6.162727e+00,
#  -6.585027e+00,  1.100000e-03, -3.900000e-01])
# eta_fiducial = np.array(np.load('epsfid.npy'))

# Cinv = faasSimulator.Cinv
# #np.save('Finv.npy', Finv) # save Finv
# mu = np.load('mu.npy') # save mu
# #np.save('Cinv.npy', Cinv) # save Cinv
# dmudt = np.load('dmudt.npy') # save dmudt


# Compressor = score.Gaussian(len(faasSimulator.data()[0,:]), np.concatenate([theta_fiducial, eta_fiducial]), 
#                             mu = mu, Cinv = Cinv, dmudt = dmudt)

# Compressor.compute_fisher()
# Finv = Compressor.Finv[0:10,0:10]

# nuisance_indices = np.arange(10,104)

# def compressor(d, compressor_args):
#     nuisances_indices = compressor_args[0]
#     return Compressor.projected_scoreMLE(d, nuisance_indices)
# compressor_args = [nuisance_indices]

## Set up the compressor
This must have the signature `compressor(data, args)` -> `np.array([n_summaries])`
In this case we are going to do _nuisance hardened_ Gaussian score compression $$\bar{\mathbf{t}}_\theta = \mathbf{t}_\theta - \mathbf{F}_{\theta\eta}\mathbf{F}^{-1}_{\eta\eta}\mathbf{t}_\eta$$ where $$\mathbf{t}_{(\theta, \eta)} = \nabla_{(\theta, \eta)}^T\boldsymbol\mu_*\mathbf{C}^{-1}(\mathbf{d}-\boldsymbol\mu_*)$$
We'll use the class `score.Gaussian`. For this we'll need some fiducial parameters, the mean its derivative at the fiducial parameters, the inverse covariance, and the inverse Fisher matrix

In [ ]:
# params = []
# for i in range(100):
#     ps = prior.draw()
#     eps = epsilon_prior.draw()
#     params.append(list(ps)+list(eps))
# #print(ps)
# preds = []
# for i in range(100):
#     print(len(preds))
#     print(i)
#     preds.append(faasSimulator.simulation(list(params[i]),seed))

#predictions = np.asarray(preds[:100]).T
#print(np.asarray(preds)[:100,:].shape)
C = np.cov(np.asarray(preds)[:100,:].T)
#print(C)
Cinv = np.linalg.inv(C)
#np.save('Cinv.npy', Cinv)
#Cinv = np.load('Cinv.npy')
#print(Cinv.shape)
#print(Cinv)

In [ ]:
def _getAplus(A):
    eigval, eigvec = np.linalg.eig(A)
    Q = np.matrix(eigvec)
    xdiag = np.matrix(np.diag(np.maximum(eigval, 0)))
    return Q*xdiag*Q.T

def _getPs(A, W=None):
    W05 = np.matrix(W**.5)
    return  W05.I * _getAplus(W05 * A * W05) * W05.I

def _getPu(A, W=None):
    Aret = np.array(A.copy())
    Aret[W > 0] = np.array(W)[W > 0]
    return np.matrix(Aret)

def nearPD(A, nit=10):
    n = A.shape[0]
    W = np.identity(n) 
# W is the matrix used for the norm (assumed to be Identity matrix here)
# the algorithm should work for any diagonal W
    deltaS = 0
    Yk = A.copy()
    for k in range(nit):
        Rk = Yk - deltaS
        Xk = _getPs(Rk, W=W)
        deltaS = Xk - Rk
        Yk = _getPu(Xk, W=W)
    return Yk

In [ ]:
theta_fiducial = np.array([8.9,7.9,10.5,7.4,-3.7,-4.6,-6.1,-6.6,0.0011, -0.39])
eta_fiducial = np.array(np.load('epsfid_faas.npy'))
#print(eta_fiducial)
mu = faasSimulator.simulation(np.concatenate([theta_fiducial, eta_fiducial]), seed)

#Cinv = faasSimulator.Cinv
#print(Cinv)
h = np.array(abs(np.concatenate([theta_fiducial, eta_fiducial])))*0.01
dmudt = faasSimulator.dmudt(np.concatenate([theta_fiducial, eta_fiducial]), h)
#print(dmudt.shape)
Compressor = score.Gaussian(len(faasSimulator.data()[0,:]), np.concatenate([theta_fiducial, eta_fiducial]), 
                            mu = mu, Cinv = Cinv, dmudt = dmudt)

Compressor.compute_fisher()

Finv = Compressor.Finv[0:10,0:10]
#Finv = np.real(F_inv)
#print(Finv)
nuisance_indices = np.arange(10,104)

def compressor(d, compressor_args):
    nuisances_indices = compressor_args[0]
    return Compressor.projected_scoreMLE(d, nuisance_indices)
compressor_args = [nuisance_indices]

## Load in the compressed data

In [ ]:
print(Finv)

compressed_data = compressor(faasSimulator.data_comp(), compressor_args)
print(compressed_data)
n_data = len(compressed_data)

## Load in data

## Save FINV, only worth doing once, make sure to load

In [ ]:
# np.save('Finv.npy', Finv) # save Finv
# np.save('mu.npy', mu) # save mu
# #np.save('Cinv.npy', Cinv) # save Cinv
# np.save('dmudt.npy', dmudt) # save dmudt

In [ ]:
# data = faasSimulator.data()[0]

## Define ensemble of NDEs

In [ ]:
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=10, n_data=n_data, n_hiddens=[50,50], n_mades=8, act_fun=tf.tanh, index=0)]
        #ndes.MixtureDensityNetwork(n_parameters=10, n_data=n_data, n_components=1, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=1),
        #ndes.MixtureDensityNetwork(n_parameters=10, n_data=n_data, n_components=2, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=2),
        #ndes.MixtureDensityNetwork(n_parameters=10, n_data=n_data, n_components=3, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=3),
        #ndes.MixtureDensityNetwork(n_parameters=10, n_data=n_data, n_components=4, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=4),
        #ndes.MixtureDensityNetwork(n_parameters=10, n_data=n_data, n_components=5, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=5)]

## Create DELFI object

In [ ]:
DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs, Finv = Finv, theta_fiducial = theta_fiducial,
                       param_limits = [np.concatenate((lower, [0,-(0.39*2)])), np.concatenate((upper, [(0.0011*2),0]))],
                       param_names = ['logKonTN', 'logKonTC', 'logKonRN', 'logKonRC', 'logKDTN', 'logKDTC', 'logKDRN', 'logKDRC', 'malpha', 'alpha0'],
                       results_dir = "results_batch_whole_newCinv/",input_normalization="fisher", restore = False, save = True, nwalkers = 500)

In [ ]:
# DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs,
#                        param_names = ['logKonTN', 'logKonTC', 'logKonRN', 'logKonRC', 'logKDTN', 'logKDTC', 'logKDRN', 'logKDRC', 'malpha', 'alpha0'],
#                        results_dir = "results_batch_whole_comp/", restore = False, save = True)

## Fisher pre-training to initialize the NDEs

In [ ]:
DelfiEnsemble.fisher_pretraining()

## Sequential Neural Likelihood

In [ ]:
n_initial = 100
n_batch = 100
n_populations = 10

DelfiEnsemble.sequential_training(simulator, compressor, n_initial, n_batch, n_populations, patience=50., save_intermediate_posteriors=False, simulator_args = simulator_args, compressor_args = compressor_args)

## Sample the learned posterior

In [ ]:
from emcee.autocorr import integrated_time
from emcee.moves import StretchMove

def emcee_sample(log_likelihood=None, x0=None, burn_in_chain=10, main_chain=10, nwalkers= 1000):

        # Set the log likelihood (default to the posterior if none given)
        if log_likelihood is None:
            log_likelihood = lambda x: DelfiEnsemble.log_posterior_stacked(x, DelfiEnsemble.data)[0]

        # Set up default x0
        if x0 is None:
            x0 = emcee.utils.sample_ball(theta_fiducial, [0.01]*8 + [0.0001] + [0.01], nwalkers)

        # Set up the sampler
        sampler = emcee.EnsembleSampler(nwalkers, 10, log_likelihood, moves=[StretchMove(a=1)])

        # Burn-in chain
        pos, prob, state, blobs = sampler.run_mcmc(x0, burn_in_chain, progress=True)
        sampler.reset()

        sampler.run_mcmc(pos, main_chain, progress=True)
        #print(sampler.acceptance_fraction)
        return sampler.flatchain


    

fisher_samples = emcee_sample(burn_in_chain=1000, main_chain=5000)

# acls = []
# for samps in posterior_samples:
#     acls.append(integrated_time(samps))
#     posterior_samples = posterior_samples[0:-1:int(np.max(acls)),:]

    
#print(posterior_samples)
np.save('fisher_newCinv.npy', fisher_samples) # save

## Alright let's plot it!
Feed it a list of `(n_samples, n_parameters)` arrays for making a triangle plot; in this case let's just plot the posterior samples.

In [ ]:
# # # posterior_samples = np.load('pst_samples_whole_comp_3rd.npy')
pst = np.unique(fisher_samples, axis=0)
#print(pst.shape)
#print(pst)
#print(posterior_samples.shape)
DelfiEnsemble.triangle_plot(samples=[pst], savefig=True, filename='fisher.pdf')

# param_names = ['logKonTN', 'logKonTC', 'logKonRN', 'logKonRC', 'logKDTN', 'logKDTC', 'logKDRN', 'logKDRC', 'malpha', 'alpha0']
# from IPython.display import display, Math
# params_med = []
# params_up = []
# params_low = []
# for i in range(10):
#     mcmc = np.percentile(posterior_samples[:, i], [16, 50, 84])
#     q = np.diff(mcmc)
#     txt = "\mathrm{{{3}}} = {0:.4f}_{{-{1:.4f}}}^{{{2:.4f}}}"
#     txt = txt.format(mcmc[1], q[0], q[1], param_names[i])
#     params_low.append(mcmc[1] - q[0])
#     params_med.append(mcmc[1])
#     params_up.append(mcmc[1] + q[1])
#     print(txt)


# plt.show()
# print(params_med)